In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import Dataset

# Load Dataset

In [2]:
df = pd.read_csv(r"C:\Users\abiav\setiment_analysis_perdication\googlestoreappreview.csv")

In [3]:
df = df[['Translated_Review', 'Sentiment']].dropna()

In [4]:
label_map = {'Positive': 1, 'Negative': 0, 'Neutral': 2}
df['label'] = df['Sentiment'].map(label_map)

In [5]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Translated_Review'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    random_state=42
)

# Bert Tokenization

In [6]:

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [7]:

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)


In [8]:

train_dataset = Dataset.from_dict({**train_encodings, 'labels': train_labels})
test_dataset = Dataset.from_dict({**test_encodings, 'labels': test_labels})


In [9]:

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
pip install --upgrade transformers


Note: you may need to restart the kernel to use updated packages.


# Traning Configuration

In [11]:

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",  
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy"
)



In [12]:

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = torch.argmax(torch.tensor(logits), dim=-1)
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    cm = confusion_matrix(labels, predictions)
    print("Confusion Matrix:\n", cm)
    return {'accuracy': acc, 'f1': f1}

In [13]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [14]:

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

C:\Users\abiav\AppData\Local\Temp\ipykernel_16460\3499421742.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:

trainer.train()


C:\Users\abiav\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.645600,0.469033,0.845324,0.852490
2,0.157100,0.395754,0.899281,0.897281
3,0.071500,0.465014,0.910072,0.908989


Confusion Matrix:
 [[ 52   2   6]
 [ 21 153   7]
 [  6   1  30]]


C:\Users\abiav\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confusion Matrix:
 [[ 47   9   4]
 [  3 175   3]
 [  3   6  28]]


C:\Users\abiav\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confusion Matrix:
 [[ 47   9   4]
 [  4 174   3]
 [  2   3  32]]


TrainOutput(global_step=417, training_loss=0.2866858794606275, metrics={'train_runtime': 3213.3153, 'train_samples_per_second': 1.034, 'train_steps_per_second': 0.13, 'total_flos': 218647250131968.0, 'train_loss': 0.2866858794606275, 'epoch': 3.0})

In [16]:

results = trainer.evaluate()
print(results)

C:\Users\abiav\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Confusion Matrix:
 [[ 47   9   4]
 [  4 174   3]
 [  2   3  32]]
{'eval_loss': 0.4650135636329651, 'eval_accuracy': 0.9100719424460432, 'eval_f1': 0.908988586167397, 'eval_runtime': 49.0925, 'eval_samples_per_second': 5.663, 'eval_steps_per_second': 0.713, 'epoch': 3.0}


# Baseline Models (TF-IDF)

In [17]:
X = df['Translated_Review']
y = df['Sentiment']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


tfidf = TfidfVectorizer(stop_words='english', max_features=500)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf = tfidf.transform(X_test)


lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_tfidf, y_train)


y_pred = lr.predict(X_test_tfidf)


print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7877697841726619

Classification Report:
               precision    recall  f1-score   support

    Negative       0.97      0.47      0.63        60
     Neutral       0.92      0.30      0.45        37
    Positive       0.76      0.99      0.86       181

    accuracy                           0.79       278
   macro avg       0.88      0.59      0.65       278
weighted avg       0.82      0.79      0.76       278



In [38]:
model_path = "./sentiment_mode"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)
print(f"Model and tokenizer saved to {model_path}")


Model and tokenizer saved to ./sentiment_mode


In [22]:
loaded_tokenizer = BertTokenizer.from_pretrained(model_path)
loaded_model = BertForSequenceClassification.from_pretrained(model_path)
loaded_model.eval()


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [39]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch

# Load model and tokenizer
model_path = r"C:\Users\abiav\setiment_analysis_perdication\saved_model"
tokenizer = BertTokenizer.from_pretrained(model_path)
model = BertForSequenceClassification.from_pretrained(model_path)
model.eval()  # Set model to evaluation mode

# Map numeric labels to sentiment
label_map = {0: "Negative", 1: "Positive", 2: "Neutral"}

while True:
    review = input("Enter a app review (or 'quit' to exit): ")
    if review.lower() == "quit":
        break

    # Tokenize input
    inputs = tokenizer(review, return_tensors="pt", truncation=True, padding=True)

    # Get model predictions
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        pred_label = torch.argmax(logits, dim=1).item()

    # Convert numeric label to sentiment
    sentiment = label_map[pred_label]
    print(f"Sentiment: {sentiment}")


Enter a app review (or 'quit' to exit):  it borining while use app but its okay


Sentiment: Positive


Enter a app review (or 'quit' to exit):  quit
